# <center>Curso de Modelagem de Dados para IA - PARTE 10</center>

<img src="image.jpg" alt="Drawing" style="width: 300px;"/>


## Aumentando os Dados
### Combinando conjuntos de dados
Os conjuntos de dados às vezes vêm de fontes diferentes, mas podem ser combinados de maneiras úteis para produzir conjuntos de dados maiores. O Pandas fornece funcionalidade substancial para mesclar, unir e concatenar dataframes de maneira semelhante aos tipos de operações de junção fornecidas pelos bancos de dados. Vamos considerar a questão da concatenação aqui.

Conforme mencionado anteriormente, os dados sobre incêndios florestais contêm várias planilhas, incluindo informações sobre incêndios em 2017 que não foram incluídos nos dados históricos agregados na planilha de 2016. Podemos ler esses dados adicionais de 2017 e concatená-los com os dados de 2016 para construir um conjunto de dados completo. Ao fazer isso, no entanto, teremos que fazer um pouco mais de limpeza de dados.

Primeiro, vamos ler a planilha de 2017 e comparar os títulos das colunas dos dataframes de 2016 e 2017. Estamos assumindo que as colunas em cada dataframe serão as mesmas, para que possamos concatenar ao longo da dimensão de linha (eixo = 0).

In [1]:
import pandas as pd

In [3]:
df16 = pd.read_excel('Fires_100.xlsx', sheet_name='2016')
df17 = pd.read_excel('data/wildfires/Fires_100.xlsx', sheet_name='2017', na_values=[''], 
                   converters= {'ALARM_DATE': pd.to_datetime, 'CONT_DATE': pd.to_datetime})
print(df17.columns)
print(df16.columns)

Index(['YEAR_', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME', 'INC_NUM',
       'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE'],
      dtype='object')
Index(['YEAR', 'STATE', 'AGENCY', 'UNIT_ID', 'FIRE_NAME', 'INC_NUM',
       'ALARM_DATE', 'CONT_DATE', 'CAUSE', 'COMMENTS', 'REPORT_AC',
       'GIS_ACRES', 'C_METHOD', 'OBJECTIVE'],
      dtype='object')


Os nomes das colunas parecem quase idênticos, mas podemos ver que a primeira coluna em df17 é "YEAR_" e a primeira em 2016 é "YEAR". Para poder concatenar esses dataframes sem problemas, teremos que renomear a coluna em df17, o que podemos fazer com o método rename no dataframe.

In [4]:
df17.rename(columns={'YEAR_': 'YEAR'}, inplace=True)

Tendo renomeado a coluna (o que fizemos inplace, ou seja, modificando o dataframe diretamente), podemos prosseguir com a concatenação para produzir um novo dataframe com todos os dados, que usaremos para análises posteriores. Observe que, como estamos concatenando para adicionar linhas adicionais ao nosso dataframe inicial, faremos isso ao longo de axis=0:

In [6]:
dffires = pd.concat((df16, df17), axis=0)

### Adicionando dados derivados
Às vezes, o conjunto de dados bruto com o qual estamos trabalhando contém as informações básicas de que precisamos para realizar uma análise de interesse, mas não de uma forma que seja imediatamente útil para nós. Nesse caso, podemos querer aumentar o conjunto de dados com novos elementos de dados derivados dos dados brutos. Como os dataframes do pandas são uma estrutura de dados facilmente extensível, é simples adicionar esses novos elementos de dados de forma muito sucinta. Ilustraremos um pouco disso no contexto dos conjuntos de dados de beisebol e incêndios florestais.

O conjunto de dados de rebatidas de beisebol contém muitas informações sobre rebatidas, mas estranhamente não contém informações sobre o número de rebatidas simples (ou seja, rebatidas de uma base nas quais o rebatedor atingiu a primeira base com segurança e permaneceu lá até o próximo rebatedor rebater a bola). No beisebol, existem 4 tipos de rebatidas:

- simples (uma base, ou 1B)
- duplas (duas bases, ou 2B)
- triplos (três bases, ou 3B)
- home runs (quatro bases, ou HR)

Como o conjunto de dados contém informações sobre o número total de acertos **H**, bem como o número de **2B, 3B** e **HR**, podemos definir uma coluna adicional nos dataframes de rebatidas e equipes para calcular o número de singles (**1B**), que é apenas **H-(2B+3B+HR)**. Uma olhada em um dos dataframes mostra que os dados **1B** foram adicionados como a última coluna.

In [8]:
import glob, os
import pandas as pd

In [9]:
def read_all_databank_core_csv(directory):
    """
    read all csv files in the specified baseball databank directory and
    populate a dictionary storing each of the tables keyed to its name
    """
    dfs = {}
    files = glob.glob('{}/*.csv'.format(directory))
    for f in files:
        d, name = os.path.split(f)
        table = os.path.splitext(name)[0]
        df = pd.read_csv(f)
        dfs[table] = df
    return dfs

bbdfs = read_all_databank_core_csv('data/baseballdatabank/core')

In [12]:
# extract a few for further processing
batting = bbdfs['Batting']
pitching = bbdfs['Pitching']
teams = bbdfs['Teams']

In [13]:
batting['1B'] = batting['H'] - batting['2B'] - batting['3B'] - batting['HR']
teams['1B'] = teams['H'] - teams['2B'] - teams['3B'] - teams['HR']
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,1B
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0,0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0,26
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0,31
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0,30
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0,25


## Exercício sobre aumento de dados: The Slash Line
Agora que você sabe como adicionar colunas de dados adicionais ao seu dataframe, talvez queira obter alguma experiência prática. Em caso afirmativo, você pode adicionar uma célular logo abaixo e trabalhar com o exercício intitulado **The Slash Line**. Este exercício permite calcular estatísticas adicionais de beisebol a partir dos dados brutos no dataframe de rebatidas. A "Slash Line" corresponde ao trio de estatísticas de rebatidas conhecidas como Batting Average (BA), On-Base Percentage (OBP) e Slugging Percentage (SLG), fundamentais para a análise de dados de beisebol moderna e podem ser facilmente calculadas com base na dados que temos em mãos. Quando terminar, prossiga para a próxima parte do tutorial.